# Solar Luminosity

In [1]:
from spice.models import IcosphereModel
from spice.spectrum import simulate_observed_flux, luminosity
from spice.spectrum.spectrum import simulate_monochromatic_luminosity
import astropy.units as u
import jax.numpy as jnp
import numpy as np
from transformer_payne import Blackbody
import matplotlib.pyplot as plt
%matplotlib inline

from jax import config
config.update('jax_platform_name', 'cpu')
config.update('jax_enable_x64', True)

Let's use a simple, blackbody model just for the testing purposes, alogside with a solar-like model.

In [2]:
bb = Blackbody()
model = IcosphereModel.construct(10000, 1., 1., [5772.], bb.parameter_names)

/Users/mjablons/Documents/stellar-mesh-integration/src/spice/models/mesh_model.py:245: UserWarning: If override_log_g is True, either parameter_names must include one of [logg,loggs,log_g,log_gs,log g,log gs,surface gravity,surface gravities,surface_gravity,surface_gravities], or log_g_index must be passed for log g to be used in the spectrum emulator.
  warnings.warn(f"If override_log_g is True, either parameter_names must include one of [" + ",".join(


In [3]:
bb_wavelengths = jnp.linspace(1, 100000, 10000)

# Bolometric luminosity of the Sun

The bolometric luminosity of the Sun is the energy that is outputted by the whole Sun. To calculate that, we need to obtain the total flux first.

In [4]:
solar_luminosity = luminosity(bb.flux, model, bb_wavelengths)
print("Sun's bolometric luminosity:", solar_luminosity, "erg/s")

Sun's bolometric luminosity: 3.8240790425307373e+33 erg/s


Let's compare that to the theoretical luminosity:

$L=4\pi R^2 \sigma T^4$

In [5]:
jnp.sum(model.areas)*u.solRad.to(u.km)**2/(4*np.pi*(u.solRad.to(u.km)**2))

Array(0.99970102, dtype=float64)

We can use this value to slightly rescale our rescult and account for the fact that the surface area of the mesh is slightly smaller than the surface area of an ideal sphere.

In [6]:
sigma = (5.67*1e-8*u.W/(u.m**2)/(u.K**4)).to(u.erg/(u.cm**2)/(u.s)/(u.K**4))

print("Theoretical luminosity of the Sun:", 0.99970102*4*np.pi*(u.solRad.to(u.km)**2)*sigma*(5772*u.K)**4, "erg/s")

Theoretical luminosity of the Sun: 3.8265937209448174e+23 erg / (s cm2) erg/s


# Solar constant

In [7]:
import astropy.units as u
intensity = simulate_observed_flux(bb.intensity, model, jnp.log10(bb_wavelengths), u.AU.to(u.pc))

In [17]:
from jax.scipy.integrate import trapezoid

trapezoid(y=intensity[:, 0], x=bb_wavelengths * 1e-8).item()*(u.erg/u.s/u.cm**2).to(u.kW/u.m**2)/0.99970102

1.3601525057337118

The result is in $\frac{kW}{m^2}$. It agrees with the literature value.